In [74]:
import numpy as np

LIGNE = 6
COLONNE = 7
a = 1
print(a)
a = COLONNE
print(a)
a = COLONNE * (1 * COLONNE + LIGNE * (0))
print(a)
a = COLONNE * (1 * COLONNE + LIGNE * (7/2 +1))
print(a)
a = COLONNE * (1 * COLONNE + LIGNE * (49/2 + 1))
print(a)
a = COLONNE * (1 * COLONNE + LIGNE * (238/3 + 21 ))
print(a)
a = COLONNE * (1 * COLONNE + LIGNE * (1120/3 + 14))
print(a)


1
7
49
238.0
1120.0
4263.0
16317.0


import numpy as np
import random
from math import *
from gameFunctions import put_data_in_tab
import matplotlib.pyplot as plt


class Bandits:
    #static Random levers bernoulli parameter
    leversRand = np.random.sample(size = 10)
    #static epsilon
    eps = 0.4
    
    def __init__(self,nbLevers,nbIt,muVals = "random",eps=0.4,algo="baseline"):
        self.nbLevers = nbLevers
        self.nbIt = nbIt
        if((type(muVals).__module__ == np.__name__) or type(muVals) == list):
            self.muVals = np.array(muVals)
        else:
            self.muVals = np.random.sample(size = nbLevers)
        self.eps = eps
        self.t = 0
        algo = algo.lower()
        if(algo == "greedy" or algo == "greedyeps" or algo == "ucb"):
            self.algo = algo
        else:
            self.algo = "baseline"
        #count the reward -> useful to calculate the regret for the loss fonction
        self.reward = 0
    
    def binaryReward(self,levers,action):
        p = random.random()
        #print("Action mu is "+str(levers[action]))
        #print("Result is "+str(p))
        if(p<levers[action]):
            return 1
        else:
            return 0
            
    def baseline(self,meanLeverRewards,nbOfLeverUse):
        a = floor(random.random()*self.nbLevers)
        reward = self.binaryReward(self.muVals,a)
        nbOfLeverUse[a] += 1
        meanLeverRewards[a] = (meanLeverRewards[a]*(nbOfLeverUse[a]-1) + reward) / nbOfLeverUse[a]
        return {"action":a,"reward":reward,"meanLeverRewards":meanLeverRewards,"nbOfLeverUse":nbOfLeverUse}
    

    def greedy(self,meanLeverRewards, nbOfLeverUse):
        leverReward = np.zeros(self.nbLevers)
        for i in range(self.nbLevers):
            leverReward[i] += self.binaryReward(self.muVals,i)
            nbOfLeverUse[i]+=1
        for i in range(self.nbLevers):
            meanLeverRewards[i] = leverReward[i] / nbOfLeverUse[i]
        a = meanLeverRewards.argmax()
        reward = self.binaryReward(self.muVals,a)
        nbOfLeverUse[a] += 1
        meanLeverRewards[a] = (meanLeverRewards[a]*(nbOfLeverUse[a]-1) + reward) / nbOfLeverUse[a]
        return {"action":a,"reward":reward,"meanLeverRewards":meanLeverRewards,"nbOfLeverUse":nbOfLeverUse}
    
    def greedyEps(self,meanLeverRewards, nbOfLeverUse):
        p = random.random()
        if(p < self.eps):
            return self.baseline(meanLeverRewards, nbOfLeverUse)
        else:
            return self.greedy(meanLeverRewards, nbOfLeverUse)

    def ucb(self,meanLeverRewards, nbOfLeverUse):
        res = meanLeverRewards + np.sqrt(2*np.log10(self.t)/nbOfLeverUse)
        a = np.argmax(res)
        reward = self.binaryReward(self.muVals,a)
        nbOfLeverUse[a] += 1
        meanLeverRewards[a] = (meanLeverRewards[a]*(nbOfLeverUse[a]-1) + reward) / nbOfLeverUse[a]
        return {"action":a,"reward":reward,"meanLeverRewards":meanLeverRewards,"nbOfLeverUse":nbOfLeverUse}
        
    def pullLever(self,meanLeverRewards,nbOfLeverUse):
        self.t += 1
        if(self.algo == "baseline"):
            return self.baseline(meanLeverRewards,nbOfLeverUse)
        elif(self.algo == "greedy"):
            return self.greedy(meanLeverRewards,nbOfLeverUse)
        elif(self.algo == "greedyeps"):
            return self.greedyEps(meanLeverRewards,nbOfLeverUse)
        elif(self.algo == "ucb"):
            return self.ucb(meanLeverRewards,nbOfLeverUse)

    def optimalReward(self):
        return self.nbIt*np.max(self.muVals)


    def run(self):
        meanLeverRewards = np.zeros(len(self.muVals))
        #Start count at 1 to avoid divide by 0
        nbOfLeverUse = np.ones(len(self.muVals))
        regretTab = []
        for i in range(self.nbIt):
            pullRes = self.pullLever(meanLeverRewards,nbOfLeverUse)
            meanLeverRewards = pullRes["meanLeverRewards"]
            nbOfLeverUse = pullRes["nbOfLeverUse"]
            self.reward += pullRes["reward"]
            #print(self.reward)
            regretTab.append(self.optimalReward() - self.reward)
        regret = self.optimalReward()-self.reward
        accuracy = (self.reward / self.optimalReward())
        return {"reward":self.reward,"optimalReward":self.optimalReward(),"regret":regret,"accuracy":accuracy,"regretTab":regretTab}
    
def writeInFile():
    fBaseline = open("Baseline1.txt","w")
    fGreedy = open("Greedy1.txt","w")
    fGreedyEps = open("GreedyEps1.txt","w")
    fUcb = open("UCB1.txt","w")
    fOptimal = open("Optimal1.txt","w")
    greedy = []
    greedyEps = []
    ucb = []
    #create a linspace
    levers = np.linspace(0.0,1.0,100,endpoint = 0)
    #shuffle values
    np.random.shuffle(levers)
    a = Bandits(len(levers),50000,algo="baseline",muVals = levers)
    print("Baseline")
    res = a.run()
    baseline = res["regretTab"]
    a = Bandits(len(levers),50000,algo="greedy",muVals = levers)
    print("Greedy")
    res = a.run()
    greedy = res["regretTab"]
    a = Bandits(len(levers),50000,algo="greedyEps",muVals = levers)
    print("GreedyEps")
    res = a.run()
    greedyEps = res["regretTab"]
    a = Bandits(len(levers),50000,algo="ucb",muVals = levers)
    print("UCB")
    res = a.run()
    ucb = res["regretTab"]

    for i in range(50000):
        fBaseline.write(str(baseline[i]) + "\n")
        fGreedy.write(str(greedy[i]) + "\n")
        fGreedyEps.write(str(greedyEps[i]) + "\n")
        fUcb.write(str(ucb[i]) + "\n")
        fOptimal.write(str(levers.max()*50000)+ "\n")
    print("Written")

    fBaseline.close()
    fGreedy.close()
    fGreedyEps.close()
    fUcb.close()
    fOptimal.close()


#writeInFile()
f1 = open("Baseline1.txt","r")
f2 = open("Greedy1.txt","r")
f3 = open("GreedyEps1.txt","r")
f4 = open("UCB1.txt","r")
tabP1 = put_data_in_tab(f1)
tabP2 = put_data_in_tab(f2)
tabP3 = put_data_in_tab(f3)
tabP4 = put_data_in_tab(f4)
x = np.linspace(1,50000,50000)
f1.close()
f2.close()
f3.close()
f4.close()
plt.plot(x,tabP1, label ="BaselineRegret")
plt.plot(x,tabP2, label ="GreedyRegret")
plt.plot(x,tabP3, label ="GreedyEpsRegret")
plt.plot(x,tabP4, label ="UCBRegret")
plt.xlabel("Time")
plt.ylabel("Regret")
plt.legend()
plt.grid(lw = 0.25)
plt.title('Regret as a function of time')
plt.savefig('Regret.png')
plt.show()
'''
#create a linspace
levers = np.linspace(0.0,1.0,100,endpoint = 0)
#shuffle values
np.random.shuffle(levers)
a = Bandits(len(levers),50000,algo="baseline",muVals = levers)
loss = a.run()
print(loss)
print(a.muVals)
'''